# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,saudarkrokur,65.7461,-19.6394,10.16,71,94,2.77,IS,1724774399
1,1,casa grande,32.8795,-111.7573,32.77,35,54,2.68,US,1724774400
2,2,bom jardim,-7.7958,-35.5872,27.94,54,41,7.39,BR,1724774401
3,3,adamstown,-25.0660,-130.1015,19.70,68,100,9.37,PN,1724774402
4,4,ribeira grande,38.5167,-28.7000,27.22,47,20,3.09,PT,1724774403


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [33]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat', 
    geo=True,
    size='Humidity', 
    frame_width=800, 
    frame_height=600,
    title='City Humidity Map',
    alpha=0.7,
    xlabel = "Longitude",
    ylabel = "Latitude", 
    hover_cols=['City', 'Country', 'Humidity'],
    #cmap='inferno',  # Use a diverging color map for better visibility of high and low humidity values
    tiles = 'OSM',  # Use a light Carto theme for the map
    color = "City"
) 

# Display the map
map_plot



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [17]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_df = city_data_df.loc[(city_data_df['Max Temp'] <= 27) & 
                               (city_data_df['Max Temp'] > 21) & 
                               (city_data_df["Cloudiness"] == 0) & 
                               (city_data_df["Wind Speed"] <= 4.5), :]
# Drop any rows with null values
#narrowed_df.dropna(inplace=True)

# Display sample data
narrowed_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,albany,42.6001,-73.9662,26.35,69,0,0.89,US,1724774260
29,29,yarmouth,41.7057,-70.2286,25.19,81,0,2.24,US,1724774431
60,60,port saint john's,-31.6229,29.5448,26.76,38,0,2.73,ZA,1724774466
126,126,newman,37.3138,-121.0208,23.91,49,0,0.89,US,1724774544
177,177,piracicaba,-22.7253,-47.6492,24.06,17,0,3.18,BR,1724774484
184,184,santa cruz,-17.8000,-63.1667,22.70,33,0,4.12,BO,1724774034
231,231,laguna,38.4210,-121.4238,24.08,55,0,0.00,US,1724774660
240,240,lewiston orchards,46.3804,-116.9754,23.40,44,0,1.54,US,1724774669
426,426,richards bay,-28.7830,32.0377,26.19,41,0,3.98,ZA,1724774879
443,443,marks,51.7111,46.7486,24.12,42,0,4.05,RU,1724774896


### Step 3: Create a new DataFrame called `hotel_df`.

In [22]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
18,albany,US,42.6001,-73.9662,69,
29,yarmouth,US,41.7057,-70.2286,81,
60,port saint john's,ZA,-31.6229,29.5448,38,
126,newman,US,37.3138,-121.0208,49,
177,piracicaba,BR,-22.7253,-47.6492,17,
184,santa cruz,BO,-17.8000,-63.1667,33,
231,laguna,US,38.4210,-121.4238,55,
240,lewiston orchards,US,46.3804,-116.9754,44,
426,richards bay,ZA,-28.7830,32.0377,41,
443,marks,RU,51.7111,46.7486,42,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [24]:
# Set parameters to search for a hotel
radius = 10000
limit = 5
categories = "accommodation.hotel" 
params = {
    "apiKey": geoapify_key,
    "limit": limit,
    "categories": categories
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary

    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
albany - nearest hotel: No hotel found
yarmouth - nearest hotel: The Inn At Cape Cod
port saint john's - nearest hotel: Outback Inn
newman - nearest hotel: No hotel found
piracicaba - nearest hotel: New Life Apart Hotel
santa cruz - nearest hotel: Aviador Hotel Boutique
laguna - nearest hotel: Holiday Inn Express & Suites
lewiston orchards - nearest hotel: Hampton Inn Lewiston
richards bay - nearest hotel: SUN1 Richards Bay
marks - nearest hotel: Руслан
tirschenreuth - nearest hotel: Hotel Seenario
el granada - nearest hotel: Beach House
saint-francois - nearest hotel: Chez Lily
tsiombe - nearest hotel: No hotel found
puerto maldonado - nearest hotel: Puerto Palmeras


,City,Country,Lat,Lng,Humidity,Hotel Name
18,albany,US,42.6001,-73.9662,69,No hotel found
29,yarmouth,US,41.7057,-70.2286,81,The Inn At Cape Cod
60,port saint john's,ZA,-31.6229,29.5448,38,Outback Inn
126,newman,US,37.3138,-121.0208,49,No hotel found
177,piracicaba,BR,-22.7253,-47.6492,17,New Life Apart Hotel
184,santa cruz,BO,-17.8000,-63.1667,33,Aviador Hotel Boutique
231,laguna,US,38.4210,-121.4238,55,Holiday Inn Express & Suites
240,lewiston orchards,US,46.3804,-116.9754,44,Hampton Inn Lewiston
426,richards bay,ZA,-28.7830,32.0377,41,SUN1 Richards Bay
443,marks,RU,51.7111,46.7486,42,Руслан


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [32]:
%%capture --no-display

# Configure the map plot
map_plot_with_hotels = hotel_df.hvplot.points(
    'Lng', 'Lat', 
    geo=True,
    size='Humidity', 
    frame_width=800, 
    frame_height=600,
    title='City Humidity Map with Nearest Hotels',
    alpha=0.7,
    xlabel = "Longitude",
    ylabel = "Latitude", 
    hover_cols=['City', 'Country', 'Humidity', 'Hotel Name'],
    #cmap='inferno',  # Use a diverging color map for better visibility of high and low humidity values
    tiles = 'OSM',  # Use a light Carto theme for the map
    color = "City"
)
# Display the map
map_plot_with_hotels

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)